In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

from sklearn.impute import SimpleImputer

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/test.csv')

In [4]:
holidays_events = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/holidays_events.csv')
oil = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/oil.csv')
stores = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/stores.csv')
trandsactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/transactions.csv')

In [ ]:
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [5]:
train['year'] = pd.to_numeric(train['date'].str[:4])
train['month'] = pd.to_numeric(train['date'].str[5:7])
train['day'] = pd.to_numeric(train['date'].str[8:10])

train['date_'] = (train['year']*10000) + (train['month']*100) + train['day']

In [6]:
test['year'] = pd.to_numeric(test['date'].str[:4])
test['month'] = pd.to_numeric(test['date'].str[5:7])
test['day'] = pd.to_numeric(test['date'].str[8:10])

test['date_'] = (test['year']*10000) + (test['month']*100) + test['day']

In [7]:
trandsactions['year'] = pd.to_numeric(trandsactions['date'].str[:4])
trandsactions['month'] = pd.to_numeric(trandsactions['date'].str[5:7])
trandsactions['day'] = pd.to_numeric(trandsactions['date'].str[8:10])

trandsactions['date_'] = (trandsactions['year']*10000) + (trandsactions['month']*100) + trandsactions['day']

In [8]:
group_flag = 'store_nbr'

In [9]:
gb = train.groupby(by=group_flag)
gb2 = trandsactions.groupby(by=group_flag)
col = trandsactions[group_flag].unique()
dump = pd.DataFrame()


for i in col:
  all_data_ = gb.get_group(i).copy()
  trandsactions_ = gb2.get_group(i).copy()

  for idx, row in trandsactions_.iterrows():
    all_data_.loc[all_data_['date_'] == row['date_'], 'transactions'] = row['transactions']

  dump = pd.concat([dump, all_data_], sort=False)
    
train = dump

In [10]:
na_check = train.isna().sum() / len(train)
na_check

id              0.000000
date            0.000000
store_nbr       0.000000
family          0.000000
sales           0.000000
onpromotion     0.000000
year            0.000000
month           0.000000
day             0.000000
date_           0.000000
transactions    0.081904
dtype: float64

In [11]:
all_data = pd.concat([train, test], sort=False)

In [43]:
na_check = all_data.isna().sum() / len(all_data)
na_check

id              0.000000
date            0.000000
store_nbr       0.000000
family          0.000000
sales           0.009412
onpromotion     0.000000
year            0.000000
month           0.000000
day             0.000000
date_           0.000000
transactions    0.059488
family_value    0.000000
isHoliday       0.000000
oil_price       0.000000
dtype: float64

In [19]:
all_data.dtypes

id                int64
date             object
store_nbr         int64
family           object
sales           float64
onpromotion     float64
year              int64
month             int64
day               int64
date_             int64
transactions    float64
dtype: object

In [13]:
group_flag = 'year'

col = all_data['year'].unique()
col2 = all_data['family'].unique()

gb = all_data.groupby(by=group_flag)

all_data['family_value'] = 0

for i in col:
  all_data_ = gb.get_group(i).copy()


  for j in col2:
    sum = all_data_.loc[(all_data_['family'] == j) & (all_data_['sales'] > 0), 'sales'].sum()
    cnt = all_data_.loc[(all_data_['family'] == j) & (all_data_['sales'] > 0), 'sales'].count()

    all_data.loc[(all_data['family'] == j) & (all_data['year'] == i), 'family_value'] = sum / cnt

all_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


,id,date,store_nbr,family,sales,onpromotion,year,month,day,date_,transactions,family_value
561,561,2013-01-01,25,AUTOMOTIVE,0.0,0.0,2013,1,1,20130101,770.0,5.947114
562,562,2013-01-01,25,BABY CARE,0.0,0.0,2013,1,1,20130101,770.0,NaN
563,563,2013-01-01,25,BEAUTY,2.0,0.0,2013,1,1,20130101,770.0,3.660048
564,564,2013-01-01,25,BEVERAGES,810.0,0.0,2013,1,1,20130101,770.0,1287.717597
565,565,2013-01-01,25,BOOKS,0.0,0.0,2013,1,1,20130101,770.0,NaN


In [14]:
group_flag = 'year'

col = all_data['year'].unique()
col2 = all_data['store_nbr'].unique()

gb = all_data.groupby(by=group_flag)


for i in col:
  all_data_ = gb.get_group(i).copy()

  for j in col2:
    sum = all_data_.loc[all_data_['store_nbr'] == j, 'transactions'].sum()
    cnt = all_data_.loc[all_data_['store_nbr'] == j, 'transactions'].count()

    all_data.loc[(all_data['transactions'].isna()) & (all_data['year'] == i) & (all_data['store_nbr'] == j), 'transactions'] = sum / cnt


for i in col:
  all_data_ = gb.get_group(i).copy()

  sum = all_data_['transactions'].sum()
  cnt = all_data_['transactions'].count()

  all_data.loc[(all_data['transactions'].isna()) & (all_data['year'] == i), 'transactions'] = sum / cnt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: i

In [ ]:
# distinguish holiday and it takes around 1min

all_data['isHoliday'] = False

for i in range(0, len(holidays_events)):
  all_data.loc[all_data['date'] == holidays_events['date'][i], 'isHoliday'] = True

In [1]:
oil['year'] = pd.to_numeric(oil['date'].str[:4])
oil['month'] = pd.to_numeric(oil['date'].str[5:7])
oil['day'] = pd.to_numeric(oil['date'].str[8:10])
oil['date_'] = (oil['year']*10000) + (oil['month']*100) + oil['day']


oil = oil[oil['dcoilwtico'].notna()]

NameError: ignored

In [29]:
# it takes around half-min

all_data['oil_price'] = 0

for idx, row in oil.iterrows():
  all_data.loc[all_data['date_'] == row['date_'], 'oil_price'] = row['dcoilwtico']

In [56]:
all_data.describe()

,id,store_nbr,sales,onpromotion,year,month,day,date_,transactions,family_value,oil_price
count,3.029400e+06,3.029400e+06,3.000888e+06,3.029400e+06,3.029400e+06,3.029400e+06,3.029400e+06,3.029400e+06,3.029400e+06,3.029400e+06,3.029400e+06
mean,1.514700e+06,2.750000e+01,3.577757e+02,3.538980e+01,2.014858e+03,6.224706e+00,1.570412e+01,2.014922e+07,1.687701e+03,3.589273e+02,4.838639e+01
std,8.745126e+05,1.558579e+01,1.101998e+03,1.020395e+02,1.355346e+00,3.374138e+00,8.797600e+00,1.351257e+04,9.283779e+02,8.145520e+02,3.746510e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.013000e+03,1.000000e+00,1.000000e+00,2.013010e+07,5.000000e+00,0.000000e+00,0.000000e+00
25%,7.573498e+05,1.400000e+01,0.000000e+00,0.000000e+00,2.014000e+03,3.000000e+00,8.000000e+00,2.014030e+07,1.069469e+03,3.703958e+00,0.000000e+00
50%,1.514700e+06,2.750000e+01,1.100000e+01,1.000000e+00,2.015000e+03,6.000000e+00,1.600000e+01,2.015050e+07,1.435000e+03,2.320949e+01,4.770000e+01
75%,2.272049e+06,4.100000e+01,1.958473e+02,1.600000e+01,2.016000e+03,9.000000e+00,2.300000e+01,2.016070e+07,1.993000e+03,2.999009e+02,9.223250e+01
max,3.029399e+06,5.400000e+01,1.247170e+05,1.277000e+03,2.017000e+03,1.200000e+01,3.100000e+01,2.017083e+07,8.359000e+03,4.661522e+03,1.106200e+02


In [ ]:
ooo = all_data[all_data['oil_price'] == 0].sum() / len(all_data)
ooo

In [50]:
cormat = all_data.corr()
cormat = cormat.index[abs(cormat['sales']) > 0]
all_data[cormat].corr()

,id,store_nbr,sales,onpromotion,year,month,day,date_,transactions,family_value,isHoliday,oil_price
id,1.000000,2.984948e-04,0.085784,0.217817,9.780918e-01,7.497339e-02,2.029427e-02,9.829371e-01,-0.027479,1.081482e-01,4.068479e-02,-4.085657e-01
store_nbr,0.000298,1.000000e+00,0.041196,0.002359,2.332439e-13,1.727664e-14,-2.137703e-14,6.252647e-14,0.180463,-8.714178e-16,-5.799331e-15,9.521310e-16
sales,0.085784,4.119605e-02,1.000000,0.755648,8.109275e-02,1.978967e-02,-1.188702e-02,8.185325e-02,0.214843,7.367926e-01,1.214959e-02,-7.625360e-02
onpromotion,0.217817,2.358618e-03,0.755648,1.000000,2.096830e-01,3.245951e-02,4.929654e-03,2.111313e-01,0.067396,8.235070e-01,1.235057e-02,-1.013006e-01
year,0.978092,2.332439e-13,0.081093,0.209683,1.000000e+00,-1.334969e-01,9.714814e-04,9.996934e-01,-0.035294,1.101227e-01,1.690913e-02,-4.065790e-01
month,0.074973,1.727664e-14,0.019790,0.032460,-1.334969e-01,1.000000e+00,7.035349e-03,-1.089259e-01,0.039132,-1.258921e-02,1.195910e-01,2.475883e-03
day,0.020294,-2.137703e-14,-0.011887,0.004930,9.714814e-04,7.035349e-03,1.000000e+00,1.801165e-03,-0.019434,1.135913e-04,-7.122672e-02,-4.135613e-03
date_,0.982937,6.252647e-14,0.081853,0.211131,9.996934e-01,-1.089259e-01,1.801165e-03,1.000000e+00,-0.034437,1.101417e-01,1.990016e-02,-4.077503e-01
transactions,-0.027479,1.804627e-01,0.214843,0.067396,-3.529432e-02,3.913153e-02,-1.943352e-02,-3.443666e-02,1.000000,-3.742401e-03,4.238282e-02,-9.100366e-02
family_value,0.108148,-8.714178e-16,0.736793,0.823507,1.101227e-01,-1.258921e-02,1.135913e-04,1.101417e-01,-0.003742,1.000000e+00,3.179474e-03,-4.522821e-02


In [ ]:
all_data.head()

,date,store_nbr,family,sales,onpromotion,idx,transactions
id,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,2013-01-01 1,NaN
1,2013-01-01,1,BABY CARE,0.0,0.0,2013-01-01 1,NaN
2,2013-01-01,1,BEAUTY,0.0,0.0,2013-01-01 1,NaN
3,2013-01-01,1,BEVERAGES,0.0,0.0,2013-01-01 1,NaN
4,2013-01-01,1,BOOKS,0.0,0.0,2013-01-01 1,NaN


In [ ]:
save_point = all_data
save_point.to_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/save_point.csv')

In [ ]:
save_point = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/save_point.csv')

In [ ]:
all_data = save_point

In [ ]:
# separate object data and non-object data

data_object = all_data.select_dtypes(include=object)
data_non_object = all_data.select_dtypes(exclude=object)

print(data_object.columns)
print(data_non_object.columns)

Index(['date', 'family', 'idx'], dtype='object')
Index(['Unnamed: 0', 'id', 'store_nbr', 'sales', 'onpromotion', 'isHoliday',
       'oil_price', 'transactions', 'year', 'month', 'day', 'date_'],
      dtype='object')


In [ ]:
all_data_index = all_data.index

dummy_data = pd.get_dummies(data_object.drop(['date'], axis=1), drop_first=True)
dummy_data.index = all_data_index

In [ ]:
all_data = pd.merge(dummy_data, data_non_object, right_index=True, left_index=True)

NameError: ignored

In [51]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

all_data.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,date_,transactions,family_value,isHoliday,oil_price
561,561,2013-01-01,25,AUTOMOTIVE,0.0,0.0,2013,1,1,20130101,770.0,4.696937,True,93.14
562,562,2013-01-01,25,BABY CARE,0.0,0.0,2013,1,1,20130101,770.0,0.000000,True,93.14
563,563,2013-01-01,25,BEAUTY,2.0,0.0,2013,1,1,20130101,770.0,2.410053,True,93.14
564,564,2013-01-01,25,BEVERAGES,810.0,0.0,2013,1,1,20130101,770.0,1107.950753,True,93.14
565,565,2013-01-01,25,BOOKS,0.0,0.0,2013,1,1,20130101,770.0,0.000000,True,93.14


In [52]:
all_data.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'year',
       'month', 'day', 'date_', 'transactions', 'family_value', 'isHoliday',
       'oil_price'],
      dtype='object')

In [53]:
col_x = ['store_nbr', 'family', 'onpromotion', 'year',
       'month', 'day', 'date_', 'transactions', 'family_value', 'isHoliday',
       'oil_price']
col_y = 'sales'

col_x

['store_nbr',
 'family',
 'onpromotion',
 'year',
 'month',
 'day',
 'date_',
 'transactions',
 'family_value',
 'isHoliday',
 'oil_price']

In [55]:
cormat = all_data.corr()

dump = cormat.index[abs(cormat['sales']) >= 0.1]
dump

Index(['sales', 'onpromotion', 'transactions', 'family_value'], dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

train_x = train[col_x]
train_y = np.log1p(train['sales'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test[col_x]

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

xgb = XGBRegressor(randome_state = 37)
xgb.fit(train_x, train_y)
#train_x.dtypes

[10:09:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


id                int64
store_nbr         int64
sales           float64
onpromotion     float64
isHoliday          bool
oil_price       float64
transactions      int64
year              int64
month             int64
day               int64
date_             int64
dtype: object

In [ ]:
param = {
    'max_depth':[2,3,4],
    'n_estimators':range(550,700,50),
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param, cv=2, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

#print(train_x.dtypes)
#print(train_y.dtypes)

grid_search.fit(train_x, train_y)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = xgb.predict(train_x)
pred_val = xgb.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

# train mae score:  0.7586943958044394
# val mae score: 0.942235584465081
# real score : 2.37214

# train mae score:  0.466127593673133
# val mae score: 0.5367417374485141
# real score : 2.47333

ValueError: ignored

In [ ]:
#xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/sample_submission.csv')

In [ ]:
#pred = grid_search.predict(test_x)
submission['sales'] = np.expm1(pred)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/submission.csv', index=False)